In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
import matplotlib.pyplot as plt
from torchinfo import summary
import numpy as np
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader

In [ ]:
class SimpleConvnet(nn.Module):
    def __init__(self):
        super(SimpleConvnet, self).__init__()
        self.conv2d = nn.Conv2d(in_channels=1,out_channels=9,kernel_size=(5,5))
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(5184,10)
        self.linear3 = nn.Linear(2000,10)
        
        
    def forward(self, x):
        relu_result = nn.functional.relu(self.conv2d(x))
        flatten_result = self.flatten(relu_result)
        linear_result1 = self.linear1(flatten_result)
#         linear_result2 = self.linear2(linear_result1)
#         linear_result3 = self.linear3(linear_result1)
        result = nn.functional.log_softmax(linear_result1, dim=1)
        return result
        
        

In [ ]:
net = SimpleConvnet()
summary(net, input_size=(1,1,28,28))

if torch.cuda.is_available():
    net.cuda()

In [ ]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 100


train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        X = X.cuda()
        y = y.cuda()
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.cuda()
            y = y.cuda()
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
train_loop(train_dataloader,net,loss_fn,optimizer)
test_loop(test_dataloader,net,loss_fn)